In [3]:
# 🚗 Montar Drive
from google.colab import drive
drive.mount('/content/drive')

# 📚 Librerías necesarias
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, log_loss
)
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# 📥 Cargar base reducida
ruta_base = "/content/drive/MyDrive/Datos/7_Base_Modelos_Predictivos_Reducida.parquet"
df = pd.read_parquet(ruta_base)
df = df.sort_values(["NIT", "Año"]).copy()

# 🏗️ Crear RQ_final y año final
df_rq_final = df.groupby("NIT")["RQ"].max().reset_index().rename(columns={"RQ": "RQ_final"})
df_last_year = df.groupby("NIT")["Año"].max().reset_index().rename(columns={"Año": "Año_final"})
df = df.merge(df_rq_final, on="NIT").merge(df_last_year, on="NIT")

# 🔢 Variables financieras
col_excluir = ['NIT', 'Año', 'DEP', 'CIIU_Letra', 'RQ', 'RQ_final', 'Año_final']
variables_financieras = [col for col in df.columns if col not in col_excluir]
ventana = 5
df_ventanas = []

for nit, grupo in tqdm(df.groupby("NIT"), desc="⏳ Construyendo stacks"):
    grupo = grupo.sort_values("Año", ascending=False)
    if grupo.shape[0] < ventana:
        continue
    año_final = grupo["Año"].max()
    grupo = grupo[grupo["Año"].between(año_final - ventana + 1, año_final)]
    if grupo.shape[0] < ventana:
        continue
    fila = {}
    for i, (_, fila_anio) in enumerate(grupo.sort_values("Año", ascending=False).iterrows()):
        for var in variables_financieras:
            fila[f"{var}_-{i}"] = fila_anio[var]
    fila["RQ_final"] = grupo["RQ_final"].iloc[0]
    df_ventanas.append(fila)

df_stack = pd.DataFrame(df_ventanas)
df_stack.replace([np.inf, -np.inf], np.nan, inplace=True)
df_stack.dropna(thresh=df_stack.shape[0] * 0.5, axis=1, inplace=True)
df_stack.fillna(df_stack.median(numeric_only=True), inplace=True)

# 🎯 Features y target
X = df_stack.drop(columns=["RQ_final"])
y = df_stack["RQ_final"]

# 🔢 Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ⚙️ Validación cruzada
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
accuracy_list, precision_list, recall_list, f1_list, auc_list, logloss_list = [], [], [], [], [], []

print("🚀 Ejecutando validación cruzada...")

for train_idx, test_idx in tqdm(kf.split(X_scaled, y), total=10):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # 📦 Base learners
    estimators = [
        ('logit', LogisticRegression(solver='lbfgs', max_iter=1000)),
        ('lgbm', LGBMClassifier(random_state=42)),
        ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42))
    ]

    # 🧠 Modelo de stacking
    stack_model = StackingClassifier(
        estimators=estimators,
        final_estimator=LogisticRegression(),
        cv=5,
        passthrough=False,
        n_jobs=-1
    )

    stack_model.fit(X_train, y_train)
    y_pred = stack_model.predict(X_test)
    y_prob = stack_model.predict_proba(X_test)[:, 1]

    accuracy_list.append(accuracy_score(y_test, y_pred))
    precision_list.append(precision_score(y_test, y_pred, zero_division=0))
    recall_list.append(recall_score(y_test, y_pred, zero_division=0))
    f1_list.append(f1_score(y_test, y_pred, zero_division=0))
    auc_list.append(roc_auc_score(y_test, y_prob))
    logloss_list.append(log_loss(y_test, y_prob))

# 📈 Resultados
print("\n📋 Resultados promedio (10 folds):")
print(f"Accuracy:   {np.mean(accuracy_list):.4f} ± {np.std(accuracy_list):.4f}")
print(f"Precision:  {np.mean(precision_list):.4f} ± {np.std(precision_list):.4f}")
print(f"Recall:     {np.mean(recall_list):.4f} ± {np.std(recall_list):.4f}")
print(f"F1-score:   {np.mean(f1_list):.4f} ± {np.std(f1_list):.4f}")
print(f"AUC:        {np.mean(auc_list):.4f} ± {np.std(auc_list):.4f}")
print(f"LogLoss:    {np.mean(logloss_list):.4f} ± {np.std(logloss_list):.4f}")

# 💾 Guardar en CSV
ruta_csv = "/content/drive/MyDrive/Resultados/resultados_modelos_stack5_RQfinal.csv"
if os.path.exists(ruta_csv):
    resumen = pd.read_csv(ruta_csv)
else:
    resumen = pd.DataFrame(columns=[
        'Base', 'Modelo', 'Naturaleza', 'Temporalidad', 'Tipo de aprendizaje',
        'Accuracy', 'Desv. Accuracy', 'Precision', 'Desv. Precision',
        'Recall', 'Desv. Recall', 'F1-score promedio', 'Desviación F1',
        'AUC', 'Desv. AUC', 'LogLoss', 'Desv. LogLoss',
        'Top 1 variable', 'Top 2 variable', 'Top 3 variable'
    ])

resumen.loc[len(resumen)] = [
    'Turismo', 'Stacking (Logit+LGBM+XGB)', 'Híbrido', 'Estática', 'Supervisado',
    round(np.mean(accuracy_list), 4), round(np.std(accuracy_list), 4),
    round(np.mean(precision_list), 4), round(np.std(precision_list), 4),
    round(np.mean(recall_list), 4), round(np.std(recall_list), 4),
    round(np.mean(f1_list), 4), f"±{np.std(f1_list):.4f}",
    round(np.mean(auc_list), 4), round(np.std(auc_list), 4),
    round(np.mean(logloss_list), 4), round(np.std(logloss_list), 4),
    None, None, None
]

resumen.to_csv(ruta_csv, index=False)
print(f"\n✅ Resultados guardados en: {ruta_csv}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


⏳ Construyendo stacks:   0%|          | 0/5770 [00:00<?, ?it/s]

🚀 Ejecutando validación cruzada...


  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11


📋 Resultados promedio (10 folds):
Accuracy:   0.8182 ± 0.0164
Precision:  0.8593 ± 0.0153
Recall:     0.9032 ± 0.0200
F1-score:   0.8805 ± 0.0109
AUC:        0.8956 ± 0.0172
LogLoss:    0.3850 ± 0.0246

✅ Resultados guardados en: /content/drive/MyDrive/Resultados/resultados_modelos_stack5_RQfinal.csv
